在开始讲解如何计算一个Java对象占用的内存大小之前, 先看几个谬论
### 一. 谬论

#### 1. 谬论一: Java不需要`sizeof()`函数, 因为java的基本类型大小是确定的
没错, java中的`int`类型在任何平台或虚拟机下都是32bit的. 但这只是从编程者的角度去看. 实际上, Java在语言层面并没有说明当int类型定义在类的属性中时应该在内存中以何种方式对齐, 或者也没有说明一个`boolean数组`就不能用一个压缩的bit向量来实现. 这些都会导致对象的size不能通过简单的计算基本类型叠加得到

#### 2. 谬论二: Java对象的size, 可以通过将其序列化成bit流后, 去看流的长度
序列化后的长度并不等与内存中占用的空间大小. 比如`String`类, 在内存中以char数组的形式出现, 每个char至少占2字节; 而序列化后, 假如使用utf8编码格式, 每个ASCII码只占用一半的空间

### 二. 什么是对象的size

对象的size是一个很难清晰界定的概念, 但是为了能开展讨论, 我们需要持有以下几个观点:
* 一个对象的大小, 近似等于其所有非静态属性的大小之和(包括其父类中定义的属性)
* 类的方法不影响对象的大小
* 类的接口不影响对象的大小  
```
实现接口, 只不过是编译器在编译期间检查类是否复写了接口的全部方法; 运行期间jvm并不会做此项检查
```
* 对象的全部size, 是一个闭包的大小. 从starting object开始, 到完整的对象图  

事实上, 基本数据类型占用的内存空间, 也不是像语言层面定义的那样永远不变, 严重依赖JVM对齐效果和优化手段. 具体可参考[do-you-know-your-data-size](https://www.javaworld.com/article/2077496/java-tip-130--do-you-know-your-data-size-.html). 一个检查数据类型所占用空间的工具如下:
```java
public class Sizeof {
    public static void main(String[] args) throws Exception {
        // Warm up all classes/methods we will use
        runGC();
        usedMemory();
        // Array to keep strong references to allocated objects
        final int count = 100000;
        Object[] objects = new Object[count];

        long heap1 = 0;
        // Allocate count+1 objects, discard the first one
        for (int i = -1; i < count; ++i) {
            Object object = null;

            // Instantiate your data here and assign it to object
//            object = new Object();
//            object = new Integer (i);
//            object = new Long (i);
            object = new String ();
            //object = new byte [128][1]

            if (i >= 0)
                objects[i] = object;
            else {
                object = null; // Discard the warm up object
                runGC();
                heap1 = usedMemory(); // Take a before heap snapshot
            }
        }
        runGC();
        long heap2 = usedMemory(); // Take an after heap snapshot:

        final int size = Math.round(((float) (heap2 - heap1)) / count);
        System.out.println("'before' heap: " + heap1 +
                ", 'after' heap: " + heap2);
        System.out.println("heap delta: " + (heap2 - heap1) +
                ", {" + objects[0].getClass() + "} size = " + size + " bytes");
        for (int i = 0; i < count; ++i) objects[i] = null;
        objects = null;
    }

    private static void runGC() throws Exception {
        // It helps to call Runtime.gc()
        // using several method calls:
        for (int r = 0; r < 4; ++r) _runGC();
    }

    private static void _runGC() throws Exception {
        long usedMem1 = usedMemory(), usedMem2 = Long.MAX_VALUE;
        for (int i = 0; (usedMem1 < usedMem2) && (i < 500); ++i) {
            s_runtime.runFinalization();
            s_runtime.gc();
            Thread.currentThread().yield();

            usedMem2 = usedMem1;
            usedMem1 = usedMemory();
        }
    }

    private static long usedMemory() {
        return s_runtime.totalMemory() - s_runtime.freeMemory();
    }

    private static final Runtime s_runtime = Runtime.getRuntime();
} // End of class
```
以上程序, 在64bit虚拟机下得到:
```
'before' heap: 1029688, 'after' heap: 2631040
heap delta: 1601352, {class java.lang.Integer} size = 16 bytes

'before' heap: 1029856, 'after' heap: 3431368
heap delta: 2401512, {class java.lang.Long} size = 24 bytes
```


### 三. 正确实现java的sizeof()
[https://www.javaworld.com/article/2077408/sizeof-for-java.html?page=2](https://www.javaworld.com/article/2077408/sizeof-for-java.html?page=2)  
[Download the source code that accompanies this article](http://images.techhive.com/downloads/idge/imported/article/jvw/2003/12/02-qa-1226-sizeof.zip)